In [ ]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plotter
import warnings
warnings.filterwarnings('ignore')

2024-07-24 05:21:18.374692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 05:21:18.374795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 05:21:18.531003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import gc
import os
import re
import numpy as np
import pandas as pd

import nltk
from nltk.util import ngrams
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.impute import SimpleImputer

In [ ]:
class config:
    root = "/kaggle/input/lmsys-chatbot-arena/"
    train_path = os.path.join(root, "train.csv")
    test_path = os.path.join(root, "test.csv")
    sample_submission_path = os.path.join(root, "sample_submission.csv")
    seed = 42
    n_splits = 10

In [ ]:
df = pd.read_csv(config.train_path)
test = pd.read_csv(config.test_path)
sample_submission = pd.read_csv(config.sample_submission_path)

In [ ]:
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

In [ ]:
test["prompt"] = test.prompt.map(lambda x: eval(x)[0])
test["response_a"] = test.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test["response_b"] = test.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

In [ ]:
# Define a function to create options based on the prompt and choices
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True

    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # Response from Model A
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # Response from Model B
                     ]
    return row

In [ ]:
df = df.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(df.head(2))  # Display the first 2 rows of df

In [ ]:
test = test.apply(make_pairs, axis=1)  # Apply the make_pairs function to each row in df
display(test.head(2))  # Display the first 2 rows of df

In [ ]:
df.encode_fail.value_counts(normalize=False)

In [ ]:
target_cols = ["winner_model_a", "winner_model_b", "winner_tie"]
target = "target"

df[target] = np.nan
for idx, t in enumerate(target_cols):
    df.loc[df[t] == 1, target] = idx
df[target] = df[target].astype("int32")

df.head()

In [ ]:
train_texts = df.options.tolist()  # Extract training texts
train_labels = df.target.tolist()

In [ ]:
from transformers import AutoTokenizer, TransfoXLModel
from transformers import BertModel, BertTokenizer
from transformers import RobertaTokenizer, RobertaModel, LongformerTokenizer, LongformerModel
from transformers import DistilBertTokenizer, DistilBertModel

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
model =  RobertaModel.from_pretrained('FacebookAI/roberta-base', output_hidden_states = True)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
DATA_COLUMN = 'options'
LABEL_COLUMN = 'target'
# The list containing all the classes (train['SECTION'].unique())
label_list = [x for x in np.unique(df['target'])]
label_list

[0, 1, 2]

In [ ]:
corpus_train = df['options'].to_numpy()
sample_labels_train = df['target'].to_numpy()

In [ ]:
input_ids = []
attention_masks = []
# max_len = 510
# For every tweet...
for tweet in corpus_train:
    encoded_dict = tokenizer.encode_plus(
                        tweet,
                        add_special_tokens = True, # Добавдяем специальные токены '[CLS]' и '[SEP]'
                        max_length =510,           # Выравниваем все предложения до одной длины и обрезаем их по max_len
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Добавляем attention_mask
                        return_tensors = 'pt',     # Возвращаем pytorch tensors.
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Преобразуем списки в тензора
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
labels = torch.tensor(sample_labels_train)

In [ ]:
# Собираем все обучающие данные в TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

# Рандомно разбиваем датасет
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

45,981 training samples
11,496 validation samples


In [ ]:
# The DataLoader нелбходимо указать размер батча
batch_size = 3

Создаем DataLoaders для тренировочных и валидационных данных

In [ ]:
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # Рандомный выбор батчей
            batch_size = batch_size )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), # Последовательный выбор батчей
            batch_size = batch_size )

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

### Загружаем BertForSequenceClassification - предварительно обученную BERT-модель с одним слоем линейным слоем для классификации

In [ ]:
model =BertForSequenceClassification.from_pretrained(
    "FacebookAI/roberta-base", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # Число лейблов
    output_attentions = True, # Нужно ли возвращать attentions weights.
    output_hidden_states = True, # Нужно ли возвращать скрытое состояние hidden-states.
)
model = model.to(device)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encode

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
epochs = 1
# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

Get_linear_schedule_with_warmup - планировщик скорости обучения, мягко увеличивает скорость обучения в начале обучения, чтобы избежать внезапных изменений, и плавно снижает ее в конце обучения для более эффективной точной настройки.

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Функция для расчета аккюраси

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten() # одномерный тензор
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    #hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Обучение

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []


total_t0 = time.time()

for epoch_i in range(0, epochs):

    #-------------------------------------------------
    #               Training

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Время каждой эпохи
    t0 = time.time()
    total_train_loss = 0
    model.train()

    # embeddings_train = torch.Tensor().to(device)
    for step, batch in enumerate(train_dataloader):
        # Распаковка данных  из dataloader.
        # `batch` содержит three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).long()

        # Оптимизатор диктует "правило обновления" - как будут изменяться параметры.
        optimizer.zero_grad() # Сбрасываем градиенты
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels, output_hidden_states = True)
        # print(output[2])

        # embeddings_train = torch.cat([embeddings_train, mean_pooling(output, b_input_mask)])


        loss = output.loss
        total_train_loss += loss.item()

        # Выполняем обратный проход для расчета градиентов.
        loss.backward()

        # Обрежьте градиенты до 1,0. Это необходимо для предотвращения проблемы "взрывающихся градиентов".
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Обновить параметры и делаем шаг
        optimizer.step()
        # Обновляем learning rate.
        scheduler.step()

    # Вычисляем average loss для всех батчей.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Время, затраченное на эпоху
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
    gc.collect()
    #--------------------------------------------------------------
    #               Validation

    print("")
    print("Running Validation...")
    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # embeddings_val = torch.Tensor().to(device)
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device).long()

        # Данный метод отключает вычисление градиента, что уменьшает расход памяти на вычисления.
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels, output_hidden_states = True)


            # embeddings_val = torch.cat([embeddings_val, mean_pooling(output, b_input_mask)])

        loss = output.loss
        total_eval_loss += loss.item()

        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
    gc.collect()
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



======== Epoch 1 / 1 ========
Training...

  Average training loss: 1.11
  Training epcoh took: 1:16:41

Running Validation...
  Accuracy: 0.35

Training complete!
Total training took 1:23:12 (h:mm:ss)


In [ ]:
model = torch.load('bert_model')